In [6]:
import os
import time
import numpy as np
import gymnasium as gym
from gymnasium import spaces

import torch as th
import torch.nn as nn
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.callbacks import BaseCallback


def softmax(x: np.ndarray, axis=-1) -> np.ndarray:
    x = x - np.max(x, axis=axis, keepdims=True)
    e = np.exp(x)
    return e / (np.sum(e, axis=axis, keepdims=True) + 1e-12)


def make_synthetic_prices(T: int, N: int, seed: int = 0) -> np.ndarray:
    """
    간단한 랜덤워크(로그수익률)로 가격 생성 (벤치마크용).
    실제 데이터로 바꾸려면 여기만 교체하면 됨.
    """
    rng = np.random.default_rng(seed)
    # 일간 로그수익률: 평균 0, 표준편차 1% 정도 (대충)
    log_rets = rng.normal(loc=0.0, scale=0.01, size=(T, N)).astype(np.float32)
    prices = np.exp(np.cumsum(log_rets, axis=0)).astype(np.float32)
    # 시작값 100으로 스케일
    prices = prices * 100.0
    return prices


class PortfolioEnv(gym.Env):
    """
    관측:
      - seq: 최근 window일의 "수익률" (shape: [window, N])
      - static: 직전 포트폴리오 비중(N) + 포트폴리오 가치(1) (shape: [N+1])
    행동:
      - raw action (shape [N]) -> softmax로 비중으로 변환 (합=1)
    보상:
      - log(1 + 포트수익) - 거래비용
    """
    metadata = {"render_modes": []}

    def __init__(
        self,
        prices: np.ndarray,
        window: int = 30,
        cost_rate: float = 0.001,  # 거래비용(턴오버 페널티 계수)
    ):
        super().__init__()
        assert prices.ndim == 2, "prices must be (T, N)"
        self.prices = prices
        self.T, self.N = prices.shape
        self.window = window
        self.cost_rate = cost_rate

        # 일간 단순수익률 r_t = P_t / P_{t-1} - 1
        self.returns = (prices[1:] / prices[:-1] - 1.0).astype(np.float32)  # shape (T-1, N)

        self.observation_space = spaces.Dict(
            {
                "seq": spaces.Box(low=-np.inf, high=np.inf, shape=(window, self.N), dtype=np.float32),
                "static": spaces.Box(low=-np.inf, high=np.inf, shape=(self.N + 1,), dtype=np.float32),
            }
        )
        self.action_space = spaces.Box(low=-5.0, high=5.0, shape=(self.N,), dtype=np.float32)

        self._t = None
        self._w_prev = None
        self._value = None

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self._t = self.window  # returns 인덱스 기준
        self._w_prev = np.ones(self.N, dtype=np.float32) / self.N
        self._value = 1.0
        return self._get_obs(), {}

    def _get_obs(self):
        # seq: 최근 window일 수익률 (returns는 (T-1, N))
        seq = self.returns[self._t - self.window : self._t]  # (window, N)
        static = np.concatenate([self._w_prev, np.array([self._value], dtype=np.float32)], axis=0)
        return {"seq": seq.astype(np.float32), "static": static.astype(np.float32)}

    def step(self, action):
        # action -> weights
        w = softmax(action.astype(np.float32), axis=0).astype(np.float32)

        # 거래비용(턴오버)
        turnover = np.sum(np.abs(w - self._w_prev)).astype(np.float32)
        cost = self.cost_rate * turnover

        # 다음날(현재 t)의 수익률 적용
        # returns[t]는 (P_{t+1}/P_t - 1)에 해당하는 느낌이라,
        # 여기서는 단순히 returns[self._t]를 "오늘->내일" 수익으로 사용
        r = self.returns[self._t]  # (N,)
        port_ret = float(np.dot(w, r))

        reward = float(np.log1p(port_ret) - cost)

        # 가치 업데이트(단순)
        self._value = float(self._value * (1.0 + port_ret - cost))
        self._w_prev = w

        self._t += 1
        terminated = (self._t >= (self.returns.shape[0] - 1))
        truncated = False

        obs = self._get_obs() if not terminated else self._get_obs()  # 종료 시에도 형태 유지
        info = {"portfolio_value": self._value, "turnover": float(turnover), "cost": float(cost), "port_ret": port_ret}
        return obs, reward, terminated, truncated, info


class FusionExtractor(BaseFeaturesExtractor):
    """
    Dict 관측에서
      - seq: LSTM으로 요약
      - static: MLP로 요약
    둘을 concat해서 feature로 반환
    """
    def __init__(self, observation_space: spaces.Dict, lstm_hidden: int = 64, static_hidden: int = 32):
        # features_dim은 내부에서 계산해서 super에 전달
        self.seq_shape = observation_space["seq"].shape  # (window, N)
        self.static_dim = observation_space["static"].shape[0]  # N+1
        window, n_assets = self.seq_shape

        # 임시로 feature dim 계산
        features_dim = lstm_hidden + static_hidden
        super().__init__(observation_space, features_dim)

        self.lstm = nn.LSTM(
            input_size=n_assets,
            hidden_size=lstm_hidden,
            num_layers=1,
            batch_first=True,
        )
        self.static_mlp = nn.Sequential(
            nn.Linear(self.static_dim, static_hidden),
            nn.ReLU(),
        )

    def forward(self, observations):
        # observations["seq"]: (batch, window, N)
        seq = observations["seq"]
        static = observations["static"]

        # LSTM output: out (batch, window, hidden), take last time step
        out, _ = self.lstm(seq)
        lstm_feat = out[:, -1, :]  # (batch, hidden)

        static_feat = self.static_mlp(static)  # (batch, static_hidden)
        fused = th.cat([lstm_feat, static_feat], dim=1)
        return fused


class SpeedCallback(BaseCallback):
    def __init__(self, report_every_rollout=True):
        super().__init__()
        self.report_every_rollout = report_every_rollout
        self.t0 = None
        self.last_t = None
        self.last_steps = None

    def _on_training_start(self) -> None:
        self.t0 = time.time()
        self.last_t = self.t0
        self.last_steps = 0

    def _on_step(self) -> bool:
        # 학습 계속 진행하라는 의미로 True 반환
        return True

    def _on_rollout_end(self) -> None:
        if not self.report_every_rollout:
            return
        now = time.time()
        steps = self.model.num_timesteps
        dt = now - self.last_t
        ds = steps - self.last_steps
        sps = ds / max(dt, 1e-9)
        possible_4h = sps * 4 * 3600

        print(
            f"[speed] {sps:8.1f} steps/s | "
            f"elapsed={now - self.t0:7.1f}s | "
            f"timesteps={steps:,} | "
            f"est_steps_in_4h≈{int(possible_4h):,}"
        )

        self.last_t = now
        self.last_steps = steps



def make_env(prices, window, cost_rate, seed):
    def _init():
        env = PortfolioEnv(prices=prices, window=window, cost_rate=cost_rate)
        env.reset(seed=seed)
        return env
    return _init


if __name__ == "__main__":
    # ===== 사용자 조건 =====
    N_ASSETS = 10
    WINDOW = 30
    # 5년 일봉 거래일 대략 252*5 = 1260 (대충)
    T_DAYS = 252 * 5 + 1  # prices 길이 (returns는 -1)
    COST_RATE = 0.001

    # ===== 데이터 (벤치마크: synthetic) =====
    prices = make_synthetic_prices(T=T_DAYS, N=N_ASSETS, seed=42)

    # ===== 병렬 환경 수 =====
    cpu = os.cpu_count() or 8
    n_envs = max(1, min(8, cpu // 2))  # 너무 과하게 늘리면 오히려 느릴 수 있음
    print(f"cpu={cpu}, n_envs={n_envs}")

    # Windows에서 SubprocVecEnv 문제 생기면 DummyVecEnv로 바꿔서 테스트 가능
    use_subproc = True

    env_fns = [make_env(prices, WINDOW, COST_RATE, seed=1000 + i) for i in range(n_envs)]
    vec_env = SubprocVecEnv(env_fns) if use_subproc else DummyVecEnv(env_fns)

    policy_kwargs = dict(
        features_extractor_class=FusionExtractor,
        features_extractor_kwargs=dict(lstm_hidden=64, static_hidden=32),
        net_arch=[128, 128],  # fusion 이후의 policy/value MLP
    )

    model = PPO(
        policy="MultiInputPolicy",
        env=vec_env,
        verbose=0,
        policy_kwargs=policy_kwargs,
        n_steps=2048 // n_envs if n_envs > 1 else 2048,  # rollout 길이
        batch_size=256,
        learning_rate=3e-4,
        n_epochs=10,
        device="auto",  # GPU 있으면 자동 사용
    )

    # ===== 테스트 러닝 =====
    # 여기 timesteps를 바꿔가며 측정하면 됨.
    TIMESTEPS = 200_000  # 먼저 20만으로 step/s 확인 추천
    print(f"Training for timesteps={TIMESTEPS:,} (window={WINDOW}, assets={N_ASSETS}, ~5y data)")
    cb = SpeedCallback()

    t_start = time.time()
    model.learn(total_timesteps=TIMESTEPS, callback=cb)
    t_end = time.time()

    total_sec = t_end - t_start
    sps_total = TIMESTEPS / max(total_sec, 1e-9)
    print(f"\nDONE: total_time={total_sec/60:.2f} min, avg_speed={sps_total:.1f} steps/s")

    # 4시간(14400초) 기준으로 가능한 steps
    est_4h = sps_total * 4 * 3600
    print(f"EST: in 4 hours you can do about {int(est_4h):,} timesteps at this speed.")


cpu=12, n_envs=6


c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\stable_baselines3\ppo\ppo.py:155: UserWarning: You have specified a mini-batch size of 256, but because the `RolloutBuffer` is of size `n_steps * n_envs = 2046`, after every 7 untruncated mini-batches, there will be a truncated mini-batch of size 254
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=341 and n_envs=6)
  warnings.warn(


Training for timesteps=200,000 (window=30, assets=10, ~5y data)
[speed]    673.1 steps/s | elapsed=    3.0s | timesteps=2,046 | est_steps_in_4h≈9,691,982
[speed]    430.3 steps/s | elapsed=    7.8s | timesteps=4,092 | est_steps_in_4h≈6,195,807
[speed]    369.3 steps/s | elapsed=   13.3s | timesteps=6,138 | est_steps_in_4h≈5,318,231
[speed]    464.0 steps/s | elapsed=   17.7s | timesteps=8,184 | est_steps_in_4h≈6,681,735
[speed]    472.4 steps/s | elapsed=   22.1s | timesteps=10,230 | est_steps_in_4h≈6,802,584
[speed]    551.1 steps/s | elapsed=   25.8s | timesteps=12,276 | est_steps_in_4h≈7,935,254
[speed]    554.9 steps/s | elapsed=   29.5s | timesteps=14,322 | est_steps_in_4h≈7,990,577
[speed]    547.0 steps/s | elapsed=   33.2s | timesteps=16,368 | est_steps_in_4h≈7,876,612
[speed]    566.6 steps/s | elapsed=   36.8s | timesteps=18,414 | est_steps_in_4h≈8,159,430
[speed]    554.4 steps/s | elapsed=   40.5s | timesteps=20,460 | est_steps_in_4h≈7,983,927
[speed]    563.6 steps/s | ela

In [ ]:
import time
import numpy as np
import gymnasium as gym
from gymnasium import spaces

import torch as th
import torch.nn as nn
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.callbacks import BaseCallback


def softmax(x: np.ndarray, axis=-1) -> np.ndarray:
    x = x - np.max(x, axis=axis, keepdims=True)
    e = np.exp(x)
    return e / (np.sum(e, axis=axis, keepdims=True) + 1e-12)


def make_synthetic_prices(T: int, N: int, seed: int = 0) -> np.ndarray:
    rng = np.random.default_rng(seed)
    log_rets = rng.normal(loc=0.0, scale=0.01, size=(T, N)).astype(np.float32)
    prices = np.exp(np.cumsum(log_rets, axis=0)).astype(np.float32) * 100.0
    return prices


class PortfolioEnv(gym.Env):
    metadata = {"render_modes": []}

    def __init__(self, prices: np.ndarray, window: int = 30, cost_rate: float = 0.001):
        super().__init__()
        assert prices.ndim == 2, "prices must be (T, N)"
        self.prices = prices
        self.T, self.N = prices.shape
        self.window = window
        self.cost_rate = cost_rate

        self.returns = (prices[1:] / prices[:-1] - 1.0).astype(np.float32)  # (T-1, N)

        self.observation_space = spaces.Dict(
            {
                "seq": spaces.Box(low=-np.inf, high=np.inf, shape=(window, self.N), dtype=np.float32),
                "static": spaces.Box(low=-np.inf, high=np.inf, shape=(self.N + 1,), dtype=np.float32),
            }
        )
        self.action_space = spaces.Box(low=-5.0, high=5.0, shape=(self.N,), dtype=np.float32)

        self._t = None
        self._w_prev = None
        self._value = None

        # 훈련 구간에서 한 에피소드가 대략 몇 step인지(=데이터 한 바퀴 길이) 확인용
        self.episode_steps = (self.returns.shape[0] - 2) - self.window
        if self.episode_steps < 1:
            raise ValueError("Not enough data for given window.")

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self._t = self.window
        self._w_prev = np.ones(self.N, dtype=np.float32) / self.N
        self._value = 1.0
        return self._get_obs(), {}

    def _get_obs(self):
        seq = self.returns[self._t - self.window : self._t]
        static = np.concatenate([self._w_prev, np.array([self._value], dtype=np.float32)], axis=0)
        return {"seq": seq.astype(np.float32), "static": static.astype(np.float32)}

    def step(self, action):
        w = softmax(action.astype(np.float32), axis=0).astype(np.float32)

        turnover = np.sum(np.abs(w - self._w_prev)).astype(np.float32)
        cost = self.cost_rate * turnover

        r = self.returns[self._t]
        port_ret = float(np.dot(w, r))

        reward = float(np.log1p(port_ret) - cost)

        self._value = float(self._value * (1.0 + port_ret - cost))
        self._w_prev = w

        self._t += 1
        terminated = (self._t >= (self.returns.shape[0] - 1))
        truncated = False

        obs = self._get_obs()
        info = {"portfolio_value": self._value, "turnover": float(turnover), "cost": float(cost), "port_ret": port_ret}
        return obs, reward, terminated, truncated, info


class FusionExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: spaces.Dict, lstm_hidden: int = 64, static_hidden: int = 32):
        self.seq_shape = observation_space["seq"].shape  # (window, N)
        self.static_dim = observation_space["static"].shape[0]
        window, n_assets = self.seq_shape

        features_dim = lstm_hidden + static_hidden
        super().__init__(observation_space, features_dim)

        self.lstm = nn.LSTM(
            input_size=n_assets,
            hidden_size=lstm_hidden,
            num_layers=1,
            batch_first=True,
        )
        self.static_mlp = nn.Sequential(
            nn.Linear(self.static_dim, static_hidden),
            nn.ReLU(),
        )

    def forward(self, observations):
        seq = observations["seq"]       # (batch, window, N)
        static = observations["static"] # (batch, N+1)

        out, _ = self.lstm(seq)
        lstm_feat = out[:, -1, :]       # (batch, hidden)
        static_feat = self.static_mlp(static)

        return th.cat([lstm_feat, static_feat], dim=1)


class SpeedCallback(BaseCallback):
    def __init__(self):
        super().__init__()
        self.t0 = None
        self.last_t = None
        self.last_steps = None

    def _on_training_start(self) -> None:
        self.t0 = time.time()
        self.last_t = self.t0
        self.last_steps = 0

    def _on_step(self) -> bool:
        return True

    def _on_rollout_end(self) -> None:
        now = time.time()
        steps = self.model.num_timesteps
        dt = now - self.last_t
        ds = steps - self.last_steps
        sps = ds / max(dt, 1e-9)

        print(f"[speed] {sps:8.1f} steps/s | elapsed={now - self.t0:7.1f}s | timesteps={steps:,}")

        self.last_t = now
        self.last_steps = steps


def main():
    # ===== 표랑 “크기” 맞추기 =====
    # Table: training data number ≈ 32220, testing ≈ 2777
    TRAIN_POINTS = 32220
    TEST_POINTS = 2777

    # 표의 Assets: 11/11/15 중 하나
    N_ASSETS = 11     # Crypto-A/B 맞춤 (원하면 15로 바꿔)
    WINDOW = 30

    # Env에서 유효 step(episode_steps)을 TRAIN_POINTS 수준으로 맞추려고
    # episode_steps = (returns_len - 2) - window
    # returns_len = prices_len - 1
    # => prices_len ≈ TRAIN_POINTS + window + 3
    PRICES_LEN_TRAIN = TRAIN_POINTS + WINDOW + 3
    PRICES_LEN_TEST  = TEST_POINTS + WINDOW + 3

    COST_RATE = 0.001

    prices_train = make_synthetic_prices(T=PRICES_LEN_TRAIN, N=N_ASSETS, seed=42)
    prices_test  = make_synthetic_prices(T=PRICES_LEN_TEST,  N=N_ASSETS, seed=43)

    train_env = PortfolioEnv(prices=prices_train, window=WINDOW, cost_rate=COST_RATE)
    test_env  = PortfolioEnv(prices=prices_test,  window=WINDOW, cost_rate=COST_RATE)

    print(f"Assets={N_ASSETS}, window={WINDOW}")
    print(f"Train episode_steps≈{train_env.episode_steps} (target~{TRAIN_POINTS})")
    print(f"Test  episode_steps≈{test_env.episode_steps} (target~{TEST_POINTS})")

    # 벤치 목적이면 env 1개로도 충분(병렬화는 오버헤드/환경에 따라 득실)
    vec_env = DummyVecEnv([lambda: train_env])

    policy_kwargs = dict(
        features_extractor_class=FusionExtractor,
        features_extractor_kwargs=dict(lstm_hidden=64, static_hidden=32),
        net_arch=[128, 128],
    )

    model = PPO(
        policy="MultiInputPolicy",
        env=vec_env,
        verbose=0,
        policy_kwargs=policy_kwargs,
        n_steps=2048,
        batch_size=256,
        learning_rate=3e-4,
        n_epochs=10,
        device="auto",
    )

    # ===== 여기 timesteps 바꿔가며 “얼마나 걸리는지” 보면 됨 =====
    TIMESTEPS = 200_000  # 먼저 20만으로 평균 steps/s 찍고, 1M/5M 시간 환산 추천
    print(f"\nTraining for timesteps={TIMESTEPS:,} ...")
    cb = SpeedCallback()

    t0 = time.time()
    model.learn(total_timesteps=TIMESTEPS, callback=cb)
    t1 = time.time()

    total_sec = t1 - t0
    sps = TIMESTEPS / max(total_sec, 1e-9)

    print(f"\nDONE: total_time={total_sec/60:.2f} min, avg_speed={sps:.1f} steps/s")

    # “저 정도 데이터 크기(Train_POINTS) 한 바퀴”가 학습(업데이트 포함) 기준으로 대충 몇 분인지
    # (주의: PPO는 rollout/update가 섞여서 ‘순수 환경 스텝’만의 시간은 아님. 그래도 비교용으로는 쓸만함.)
    est_epoch_min = (TRAIN_POINTS / max(sps, 1e-9)) / 60.0
    print(f"EST: ~{TRAIN_POINTS} steps (≈one train pass) takes about {est_epoch_min:.2f} min at this training speed.")

    for target in [1_000_000, 5_000_000, 10_000_000]:
        est_min = (target / max(sps, 1e-9)) / 60.0
        print(f"EST: {target:,} timesteps -> ~{est_min:.1f} min")


if __name__ == "__main__":
    main()


Assets=11, window=30
Train episode_steps≈32220 (target~32220)
Test  episode_steps≈2777 (target~2777)

Training for timesteps=200,000 ...
[speed]    341.7 steps/s | elapsed=    6.0s | timesteps=2,048
[speed]    262.1 steps/s | elapsed=   13.8s | timesteps=4,096
[speed]    294.8 steps/s | elapsed=   20.8s | timesteps=6,144
[speed]    299.9 steps/s | elapsed=   27.6s | timesteps=8,192
[speed]    235.0 steps/s | elapsed=   36.3s | timesteps=10,240
[speed]    195.9 steps/s | elapsed=   46.8s | timesteps=12,288
[speed]    177.8 steps/s | elapsed=   58.3s | timesteps=14,336
[speed]    307.0 steps/s | elapsed=   64.9s | timesteps=16,384
[speed]    284.9 steps/s | elapsed=   72.1s | timesteps=18,432
[speed]    290.9 steps/s | elapsed=   79.2s | timesteps=20,480
[speed]    300.1 steps/s | elapsed=   86.0s | timesteps=22,528
[speed]    316.0 steps/s | elapsed=   92.5s | timesteps=24,576
[speed]    306.5 steps/s | elapsed=   99.2s | timesteps=26,624
[speed]    329.9 steps/s | elapsed=  105.4s | ti

In [2]:
import time
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F


# =========================
# 0) Synthetic prices (PPO 때랑 동일)
# =========================
def make_synthetic_prices(T: int, N: int, seed: int = 0) -> np.ndarray:
    rng = np.random.default_rng(seed)
    log_rets = rng.normal(loc=0.0, scale=0.01, size=(T, N)).astype(np.float32)
    prices = np.exp(np.cumsum(log_rets, axis=0)).astype(np.float32) * 100.0
    return prices


def price_relatives_from_prices(prices: np.ndarray) -> np.ndarray:
    # p_t = close_t / close_{t-1}  -> (T-1, N)
    p = prices[1:] / (prices[:-1] + 1e-12)
    return p.astype(np.float32)


# =========================
# 1) Models: "feature 추출(LSTM)" + "정책"
# =========================
class Predictor(nn.Module):
    """
    과거 window 수익률 -> 다음 price relative 힌트(p_hat) 생성
    (논문 구조를 단순화해서 LSTM으로 구현)
    """
    def __init__(self, n_assets: int, hidden: int = 64):
        super().__init__()
        self.lstm = nn.LSTM(input_size=n_assets, hidden_size=hidden, batch_first=True)
        self.head = nn.Linear(hidden, n_assets)

    def forward(self, r_hist):  # (B, window, N)
        out, _ = self.lstm(r_hist)
        h = out[:, -1, :]
        pred = self.head(h)
        p_hat = torch.exp(0.01 * pred)  # 양수 보장
        return p_hat


class PolicyNet(nn.Module):
    """
    (과거 window 수익률 feature + prev_w + p_hat) -> 현재 포트폴리오 비중 w
    cash(현금) 1개를 softmax에 같이 포함
    """
    def __init__(self, n_assets: int, hidden_lstm: int = 64, hidden_mlp: int = 64, use_cash: bool = True):
        super().__init__()
        self.use_cash = use_cash
        self.lstm = nn.LSTM(input_size=n_assets, hidden_size=hidden_lstm, batch_first=True)

        prev_dim = n_assets + (1 if use_cash else 0)
        pred_dim = n_assets + (1 if use_cash else 0)

        self.mlp = nn.Sequential(
            nn.Linear(hidden_lstm + prev_dim + pred_dim, hidden_mlp),
            nn.ReLU(),
            nn.Linear(hidden_mlp, prev_dim),  # logits
        )

    def forward(self, r_hist, prev_w, p_hat):
        out, _ = self.lstm(r_hist)
        h = out[:, -1, :]

        if self.use_cash:
            cash_pred = torch.ones((p_hat.size(0), 1), device=p_hat.device, dtype=p_hat.dtype)
            p_hat = torch.cat([cash_pred, p_hat], dim=1)  # (B, 1+N)

        x = torch.cat([h, prev_w, p_hat], dim=1)
        logits = self.mlp(x)
        w = F.softmax(logits, dim=1)
        return w


# =========================
# 2) "논문 스타일" 결정론 백테스트 + 학습 목적함수
# =========================
def turnover_cost(prev_w, w, fee_rate: float):
    return fee_rate * torch.sum(torch.abs(w - prev_w), dim=1)  # (B,)


def rollout_objective(
    p_rel_seq,                 # (B, window+horizon, N)
    window: int,
    horizon: int,
    predictor: nn.Module,
    policy: nn.Module,
    fee_rate: float = 0.0025,
    use_cash: bool = True,
):
    """
    feature 추출(LSTM) + 포트폴리오 비중 산출 + 결정론 수익/비용 계산을
    horizon 동안 굴려서 누적 log return 평균을 최대화(= objective 최대화)
    """
    B, L, N = p_rel_seq.shape
    device = p_rel_seq.device

    r_seq = p_rel_seq - 1.0  # (B, L, N)

    dim_w = N + (1 if use_cash else 0)
    prev_w = torch.ones((B, dim_w), device=device) / dim_w

    logrets = []

    for t in range(horizon):
        # (1) feature 추출 입력
        r_hist = r_seq[:, t:t + window, :]              # (B, window, N)

        # (2) predictor: p_hat
        p_hat = predictor(r_hist)                        # (B, N)

        # (3) policy: weights
        w = policy(r_hist, prev_w, p_hat)                # (B, dim_w)

        # (4) 다음 시점 실제 price relative로 수익 계산(결정론)
        p_next = p_rel_seq[:, t + window, :]             # (B, N)
        if use_cash:
            cash = torch.ones((B, 1), device=device)
            p_next = torch.cat([cash, p_next], dim=1)    # (B, 1+N)

        # 거래비용
        cost = turnover_cost(prev_w, w, fee_rate=fee_rate)
        u = 1.0 - cost

        gross = torch.sum(p_next * w, dim=1)             # (B,)
        step_logret = torch.log(torch.clamp(u * gross, min=1e-12))
        logrets.append(step_logret)

        prev_w = w

    logrets = torch.stack(logrets, dim=1)                # (B, horizon)
    return torch.mean(torch.sum(logrets, dim=1))         # scalar


# =========================
# 3) "강화학습(학습루프)만" 시간 측정 러너
# =========================
def run_rl_training_only_timebench(
    prices_train: np.ndarray,
    window: int = 31,
    horizon: int = 32,
    batch_size: int = 32,
    iterations: int = 80_000,     # 논문 설정
    bench_iters: int = 2000,      # 먼저 이만큼만 돌려서 it/s 측정
    warmup_iters: int = 100,      # CUDA 워밍업
    fee_rate: float = 0.0025,
    lr: float = 3e-4,
    seed: int = 42,
    device: str = "cuda" if torch.cuda.is_available() else "cpu",
):
    assert prices_train.ndim == 2 and np.isfinite(prices_train).all()

    np.random.seed(seed)
    torch.manual_seed(seed)

    if device.startswith("cuda") and torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.benchmark = True

    p_train = price_relatives_from_prices(prices_train)  # (T-1, N)
    Ttr, N = p_train.shape
    assert Ttr > window + horizon + 2, "train data too short: window/horizon 줄여"

    predictor = Predictor(n_assets=N, hidden=64).to(device)
    policy    = PolicyNet(n_assets=N, hidden_lstm=64, hidden_mlp=64, use_cash=True).to(device)
    optim = torch.optim.Adam(list(predictor.parameters()) + list(policy.parameters()), lr=lr)

    # 배치 샘플링 (학습 파트에 포함되는 "피처 입력" 구성)
    max_s = Ttr - (window + horizon) - 1
    assert max_s > 0, "train data too short for sampling"
    p_train_np = p_train  # numpy view

    def sample_batch():
        idx = np.random.randint(0, max_s, size=(batch_size,))
        batch = np.stack([p_train_np[s:s + window + horizon, :] for s in idx], axis=0)  # (B, window+horizon, N)
        x = torch.from_numpy(batch).to(device=device, dtype=torch.float32)
        return x

    def sync():
        if device.startswith("cuda") and torch.cuda.is_available():
            torch.cuda.synchronize()

    predictor.train(); policy.train()

    print("\n====================")
    print("RL TRAINING ONLY (feature(LSTM) + rollout + backward + step)")
    print("====================")
    print(f"device={device} | assets={N}")
    print(f"train_prices={prices_train.shape} | p_rel={p_train.shape}")
    print(f"window={window} horizon={horizon} batch={batch_size} fee={fee_rate}")
    print(f"warmup_iters={warmup_iters} | bench_iters={bench_iters} | target_iterations={iterations}")

    # ---- warmup ----
    for _ in range(warmup_iters):
        p_seq = sample_batch()
        obj = rollout_objective(p_seq, window, horizon, predictor, policy, fee_rate=fee_rate, use_cash=True)
        loss = -obj
        optim.zero_grad(set_to_none=True)
        loss.backward()
        optim.step()
    sync()

    # ---- measured bench (구간별 breakdown 포함) ----
    t_sample = 0.0
    t_forward = 0.0
    t_backward = 0.0
    t_step = 0.0

    sync()
    t0_all = time.perf_counter()

    for it in range(bench_iters):
        # 1) sample
        t0 = time.perf_counter()
        p_seq = sample_batch()
        sync()
        t_sample += time.perf_counter() - t0

        # 2) forward+rollout (feature 추출 포함)
        t0 = time.perf_counter()
        obj = rollout_objective(p_seq, window, horizon, predictor, policy, fee_rate=fee_rate, use_cash=True)
        loss = -obj
        sync()
        t_forward += time.perf_counter() - t0

        # 3) backward
        t0 = time.perf_counter()
        optim.zero_grad(set_to_none=True)
        loss.backward()
        sync()
        t_backward += time.perf_counter() - t0

        # 4) step
        t0 = time.perf_counter()
        optim.step()
        sync()
        t_step += time.perf_counter() - t0

        if (it + 1) % 200 == 0:
            elapsed = time.perf_counter() - t0_all
            it_s = (it + 1) / max(elapsed, 1e-12)
            eta_sec = iterations / max(it_s, 1e-12)
            print(f"[bench] {it+1}/{bench_iters}  speed={it_s:.3f} it/s  ETA(80k)≈{eta_sec/3600:.2f} h")

    sync()
    t1_all = time.perf_counter()
    total = t1_all - t0_all

    it_s = bench_iters / max(total, 1e-12)
    est_sec = iterations / max(it_s, 1e-12)

    def per_iter_ms(x): return 1000.0 * x / max(bench_iters, 1)

    print("\n====================")
    print("RESULT (time only)")
    print("====================")
    print(f"speed = {it_s:.3f} iterations/s")
    print(f"EST: {iterations:,} iterations -> {est_sec/60:.1f} min ({est_sec/3600:.2f} h)")
    print(
        "per-iter avg(ms): "
        f"sample={per_iter_ms(t_sample):.2f} | "
        f"forward+rollout(feature포함)={per_iter_ms(t_forward):.2f} | "
        f"backward={per_iter_ms(t_backward):.2f} | "
        f"step={per_iter_ms(t_step):.2f} | "
        f"total={per_iter_ms(total):.2f}"
    )


# =========================
# 4) main: PPO 때랑 같은 데이터 크기로 생성 후 "학습시간만" 측정
# =========================
def main():
    # (PPO 코드에서 쓰던 크기 그대로)
    TRAIN_POINTS = 32220
    N_ASSETS = 11
    PPO_WINDOW = 30

    PRICES_LEN_TRAIN = TRAIN_POINTS + PPO_WINDOW + 3
    prices_train = make_synthetic_prices(T=PRICES_LEN_TRAIN, N=N_ASSETS, seed=42)

    # 논문 기본: window=31, horizon=32, batch=32, iterations=80k
    run_rl_training_only_timebench(
        prices_train=prices_train,
        window=31,
        horizon=32,
        batch_size=32,
        iterations=80_000,
        bench_iters=2000,
        warmup_iters=100,
        fee_rate=0.0025,
        lr=3e-4,
        seed=42,
        device="cuda" if torch.cuda.is_available() else "cpu",
    )


if __name__ == "__main__":
    main()



RL TRAINING ONLY (feature(LSTM) + rollout + backward + step)
device=cpu | assets=11
train_prices=(32253, 11) | p_rel=(32252, 11)
window=31 horizon=32 batch=32 fee=0.0025
warmup_iters=100 | bench_iters=2000 | target_iterations=80000
[bench] 200/2000  speed=1.463 it/s  ETA(80k)≈15.18 h
[bench] 400/2000  speed=1.463 it/s  ETA(80k)≈15.19 h
[bench] 600/2000  speed=1.460 it/s  ETA(80k)≈15.22 h
[bench] 800/2000  speed=1.459 it/s  ETA(80k)≈15.23 h
[bench] 1000/2000  speed=1.461 it/s  ETA(80k)≈15.21 h
[bench] 1200/2000  speed=1.462 it/s  ETA(80k)≈15.20 h
[bench] 1400/2000  speed=1.464 it/s  ETA(80k)≈15.18 h
[bench] 1600/2000  speed=1.464 it/s  ETA(80k)≈15.18 h
[bench] 1800/2000  speed=1.466 it/s  ETA(80k)≈15.16 h
[bench] 2000/2000  speed=1.467 it/s  ETA(80k)≈15.15 h

RESULT (time only)
speed = 1.467 iterations/s
EST: 80,000 iterations -> 909.2 min (15.15 h)
per-iter avg(ms): sample=0.19 | forward+rollout(feature포함)=265.03 | backward=415.17 | step=1.49 | total=681.88
